# 1.将一天的sungap当作一个月的sungap(sl_id,date,hour,sundur)

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# 读取数据文件到 DataFrame
input_file = r'C:\Users\wac\Desktop\yanzheng1\results\sungap.txt'
data = pd.read_csv(input_file, sep=',', header=None, names=['id', '日期', '小时', '日照时长'])

# 将日期列转换为datetime类型
data['日期'] = pd.to_datetime(data['日期'])

# 创建一个存储结果数据的列表
result_data = []

# 遍历每个id的分组
for id, group in tqdm(data.groupby('id'), desc="Processing IDs", unit="ID"):
    # 遍历每个月的数据
    for date, sub_group in group.groupby(group['日期'].dt.to_period('M')):
        # 获取每个月15号的数据
        month_data = sub_group.copy()
        month_date = month_data['日期'].iloc[0]

        # 生成该月的每一天的日期
        days_in_month = pd.date_range(month_date.replace(day=1), month_date + pd.offsets.MonthEnd(0))
        
        # 复制15号的数据，作为该月整月的数据
        replicated_data = pd.concat([month_data.assign(日期=day) for day in days_in_month])
        
        # 添加到结果数据中
        result_data.append(replicated_data)

# 合并所有结果数据
final_data = pd.concat(result_data, ignore_index=True)

# 将合并后的数据保存为TXT文件
output_file = r'C:\Users\wac\Desktop\yanzheng1\results\01_results1_allyear.csv'
final_data.to_csv(output_file, header=None, index=False, sep=',')

# 打印提示信息
print(f"处理后的数据已保存到：{output_file}")


Processing IDs:   0%|          | 0/30 [00:00<?, ?ID/s]

Processing IDs: 100%|██████████| 30/30 [00:02<00:00, 12.00ID/s]


处理后的数据已保存到：C:\Users\wac\Desktop\yanzheng1\results\01_results1_allyear.csv


# 2.在尾部添加上气象站真实总辐射值、直接辐射、散射辐射、天顶角
# (sl_id,date,hour,sundur,GHI,DHI,DIR,Zenith)

In [10]:

import pandas as pd

# 读取CSV文件
csv_file_path = r'C:\Users\wac\Desktop\yanzheng1\results\02_radiation.csv'
data = pd.read_csv(csv_file_path, header=None)

# 确保数据有4745行
if len(data) != 4745:
    raise ValueError(f"CSV文件的行数({len(data)})与预期的4745不符")

#这里乘的是points的数量
multiplied_data = pd.concat([data] * 30, ignore_index=True)

# 写入新的CSV文件
output_file_path = r'C:\Users\wac\Desktop\yanzheng1\results\02_radiation_allyear.csv'
multiplied_data.to_csv(output_file_path, index=False, header=False)

print(f"数据已成功写入到 {output_file_path}")


数据已成功写入到 C:\Users\wac\Desktop\yanzheng1\results\02_radiation_allyear.csv


In [11]:
import pandas as pd
from tqdm import tqdm

# 文件路径
csv1_path = r'C:\Users\wac\Desktop\yanzheng1\results\01_results1_allyear.csv'
csv2_path = r'C:\Users\wac\Desktop\yanzheng1\results\02_radiation_allyear.csv'
output_csv_path = r'C:\Users\wac\Desktop\yanzheng1\results\01_results1_allyear_radiation.csv'

import pandas as pd
from tqdm import tqdm
# 拼接到其他csv文件里
# 读取第一个CSV文件
df1 = pd.read_csv(csv1_path)

# 读取第二个CSV文件
df2 = pd.read_csv(csv2_path)

# 合并两个DataFrame，使用 tqdm 显示进度条
with tqdm(total=len(df2.columns), desc="Merging data") as pbar:
    merged_df = pd.concat([df1, df2], axis=1)
    pbar.update(len(df2.columns))

# 将合并后的数据写入新的CSV文件
merged_df.to_csv(output_csv_path, index=False)

print("合并完成，并已保存到:", output_csv_path)

Merging data: 100%|██████████| 4/4 [00:00<00:00, 1000.31it/s]


合并完成，并已保存到: C:\Users\wac\Desktop\yanzheng1\results\01_results1_allyear_radiation.csv


# 3.在尾部添加上散射占比um的值
# # (sl_id,date,hour,sundur,GHI,DHI,DIR,Zenith,um)

In [17]:
# 这个代码跳过，不运行
import pandas as pd
import os
# 将汇总的UM_SM分成13份
# 读取Excel文件
file_path = r'C:\Users\wac\Desktop\yanzheng3\img\result\0_所有街景图片的天空视域图_UM_SM.xlsx'
df = pd.read_excel(file_path, header=None, names=['ID', 'UM'])

# 创建保存CSV文件的目录
output_dir =r'0_所有街景图片的天空视域图_UM_SM'
os.makedirs(output_dir, exist_ok=True)

# 定义每个CSV文件的数据范围
chunk_size = 1000

# 分割数据并保存为多个CSV文件
for i in range(13):
    start_id = i * chunk_size + 1
    end_id = (i + 1) * chunk_size
    if i == 12:  # 最后一部分
        chunk_df = df[df['ID'] >= start_id]
    else:
        chunk_df = df[(df['ID'] >= start_id) & (df['ID'] <= end_id)]
    
    # 保存到CSV文件
    chunk_df.to_csv(os.path.join(output_dir, f'chunk_{i+1}.csv'), index=False)

print("完成分割并保存为CSV文件。")


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\paper\\01_studyarea\\03_太阳能辐射估计\\03_02_跑全部的结果\\1_计算日照时长\\results\\0_所有街景图片的天空视域图_UM_SM.xlsx'

In [12]:
import pandas as pd
# 每个id对应得UM值，扩大为一年得值，也就是4745倍
# 读取CSV文件，假设第二列是目标数据
input_file_path = r'C:\Users\wac\Desktop\yanzheng1\results\03_um1.csv'
  # 请替换为你的CSV文件路径
df = pd.read_csv(input_file_path, header=None, usecols=[1], names=['UM'])

# 初始化一个空的DataFrame用于存储扩展的数据
expanded_df = pd.DataFrame()

# 将第二列的每一行数据复制为4745个相同值
for value in df['UM']:
    repeated_values = [value] * 4745
    expanded_df = pd.concat([expanded_df, pd.DataFrame(repeated_values, columns=['UM'])], ignore_index=True)

# 保存到新的CSV文件，不包含header
output_file_path = r'C:\Users\wac\Desktop\yanzheng1\results\03_um1_allyear.csv'
expanded_df.to_csv(output_file_path, index=False, header=False)

print("完成扩展并保存为新的CSV文件。")


完成扩展并保存为新的CSV文件。


In [8]:
# 这个代码跳过，不运行
import pandas as pd

# 文件路径
file_path = r'C:\Users\wac\Desktop\yanzheng3\img\result\01_results1_allyear_radiation.csv'

try:
    # 分块读取CSV文件
    chunk_size = 10000  # 每次读取10000行
    num_rows = 0
    
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        num_rows += len(chunk)
    
    print(f"CSV文件的行数为: {num_rows}")
except Exception as e:
    print(f"读取CSV文件时出错: {e}")


CSV文件的行数为: 4744


In [14]:
import pandas as pd
# 将扩大后得um添加到合并csv文件中
# 读取第一个CSV文件
file1_path = r'C:\Users\wac\Desktop\yanzheng1\results\01_results1_allyear_radiation.csv'  # 请替换为你的第一个CSV文件路径
df1 = pd.read_csv(file1_path)

# 读取第二个CSV文件
file2_path = r'C:\Users\wac\Desktop\yanzheng1\results\03_um1_allyear.csv' # 请替换为你的第二个CSV文件路径
df2 = pd.read_csv(file2_path)

# 检查两个文件的行数是否相同
if len(df1) != len(df2):
    raise ValueError("两个CSV文件的行数不相同，无法合并。")

# 合并两个DataFrame
merged_df = pd.concat([df1, df2], axis=1)

# 保存合并后的CSV文件
output_file_path = r'C:\Users\wac\Desktop\yanzheng1\results\01_results1_allyear_radiation_um.csv'  # 请替换为你要保存的CSV文件路径
merged_df.to_csv(output_file_path, index=False)

print("完成合并并保存为新的CSV文件。")


完成合并并保存为新的CSV文件。


# 4.计算总辐射值、直接辐射值、散射辐射值
# (sl_id,date,hour,sundur,GHI,DHI,DIR,Zenith,um,GHI_考虑遮挡,DHI_考虑遮挡,DIR_考虑遮挡)

In [16]:
import pandas as pd
from tqdm import tqdm
import numpy as np

# 读取并处理CSV文件的函数
def process_csv(file_path, output_file_path, chunksize=100000):
    # 计算总行数
    total_rows = sum(1 for _ in open(file_path))
    chunks = pd.read_csv(file_path, chunksize=chunksize, header=None)
    
    with tqdm(total=total_rows, desc="Processing") as pbar:
        for chunk in chunks:
            # 将角度从度转换为弧度
            radians = np.radians(chunk.iloc[:, 7].astype(float))
            # 计算余弦值
            cos_values = np.cos(radians)
            
            # 计算第11列的值：=F5*D5*I5+G5*J5
            chunk['J'] = chunk.iloc[:, 5] * chunk.iloc[:, 3] * cos_values + chunk.iloc[:, 6] * chunk.iloc[:, 8]
            chunk['K'] = chunk.iloc[:, 5] * chunk.iloc[:, 3]
            chunk['L'] = chunk.iloc[:, 6] * chunk.iloc[:, 8]
            
            # 追加到输出文件，不包含header
            chunk.to_csv(output_file_path, mode='a', index=False, header=False)
            pbar.update(len(chunk))

# 文件路径
file_path = r'C:\Users\wac\Desktop\yanzheng1\results\01_results1_allyear_radiation_um.csv'  # 请替换为你的CSV文件路径
output_file_path = r'C:\Users\wac\Desktop\yanzheng1\results\01_results1_allyear_radiation_um_predict1.csv'  # 请替换为你要保存的CSV文件路径

# 处理CSV文件并显示进度
process_csv(file_path, output_file_path)

print("计算完成并保存为新的CSV文件。")


Processing: 100%|██████████| 142350/142350 [00:00<00:00, 193947.61it/s]

计算完成并保存为新的CSV文件。


# 5.计算太阳能电池板上的有效辐射值
# (sl_id,date,hour,sundur,GHI,DHI,DIR,Zenith,um,GHI_考虑遮挡,DHI_考虑遮挡,DIR_考虑遮挡,GHI_电池板上倾斜表面)

In [2]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import math
# 读取并处理CSV文件的函数
def process_csv(file_path, output_file_path, chunksize=100000):
    # 计算总行数
    total_rows = sum(1 for _ in open(file_path))
    chunks = pd.read_csv(file_path, chunksize=chunksize, header=None)
    
    with tqdm(total=total_rows, desc="Processing") as pbar:
        for chunk in chunks:
            # 将角度从度转换为弧度
            radians2 = np.radians((chunk.iloc[:, 7]-17).astype(float))
            # 计算余弦值
            cos_values2 = np.cos(radians2)
            
            # 计算第11列的值：=F5*D5*I5+G5*J5
            chunk['M'] = 0.98*(chunk.iloc[:, 10]* cos_values2 + chunk.iloc[:, 9]*0.2*(1-math.cos(math.radians(17)))/2 + chunk.iloc[:, 11]*(1+math.cos(math.radians(17)))/2 + chunk.iloc[:, 9]*(0.012*(chunk.iloc[:, 7]-0.04))*(1-math.cos(math.radians(17)))/2)
            # 追加到输出文件，不包含header

            chunk.to_csv(output_file_path, mode='a', index=False, header=False)
            pbar.update(len(chunk))

# 文件路径
file_path =  r'C:\Users\wac\Desktop\yanzheng3\img\result\01_results1_allyear_radiation_um_predict1.csv'  # 请替换为你的CSV文件路径
output_file_path =  r'C:\Users\wac\Desktop\yanzheng3\img\result\01_results1_allyear_radiation_um_predict2.csv'  # 请替换为你要保存的CSV文件路径

# 处理CSV文件并显示进度
process_csv(file_path, output_file_path)

print("计算完成并保存为新的CSV文件。")


Processing: 100%|██████████| 4745000/4745000 [00:26<00:00, 179249.26it/s]

计算完成并保存为新的CSV文件。


# 6. 计算日均辐射值和逐小时辐射值
# (sl_id,date,hour,sundur,GHI,DHI,DIR,Zenith,um,GHI_考虑遮挡,DHI_考虑遮挡,DIR_考虑遮挡,GHI_电池板上倾斜表面)

In [16]:
import pandas as pd

# 读取文件
file_path =  r'D:\paper\01_studyarea\03_太阳能辐射估计\03_02_跑全部的结果\5_所有点\folder_all\right.csv'

# 读取数据到DataFrame
data = pd.read_csv(file_path, header=None, names=['sl_id', 'date', 'hour', 'sundur','GHI','DHI','DIF','Zenith','um', 'GHI_hor', 'DHI_hor', 'DIF_hor','GHI_pv'])

# 第一部分：计算日均辐射值、日照时长
result1 = data.groupby('sl_id').agg({'sundur': 'sum', 'GHI_hor': 'sum', 'DHI_hor': 'sum', 'DIF_hor': 'sum','GHI_pv':'sum'}) / 365

# 第二部分：计算小时平均辐射值
result_df = pd.DataFrame(columns=['sl_id', '6_hor', '7_hor', '8_hor', '9_hor', '10_hor', '11_hor', '12_hor', '13_hor', '14_hor', '15_hor', '16_hor', '17_hor', '18_hor','6_pv', '7_pv', '8_pv', '9_pv', '10_pv', '11_pv', '12_pv', '13_pv', '14_pv', '15_pv', '16_pv', '17_pv', '18_pv'])

for id_value in data['sl_id'].unique():
    id_data = data[data['sl_id'] == id_value]
    avg_radiation_per_hour = []
    avg_radiation_per_hour2 = []
    for hour in range(6, 19):
        avg_radiation = id_data[(id_data['hour'] == hour)]['GHI_hor'].mean()
        avg_radiation2 = id_data[(id_data['hour'] == hour)]['GHI_pv'].mean()
        avg_radiation_per_hour.append(avg_radiation)
        avg_radiation_per_hour2.append(avg_radiation2)
    result_df = result_df.append({'sl_id': id_value, '6_hor': avg_radiation_per_hour[0], '7_hor': avg_radiation_per_hour[1], 
                                  '8_hor': avg_radiation_per_hour[2], '9_hor': avg_radiation_per_hour[3], '10_hor': avg_radiation_per_hour[4], 
                                  '11_hor': avg_radiation_per_hour[5], '12_hor': avg_radiation_per_hour[6], '13_hor': avg_radiation_per_hour[7], 
                                  '14_hor': avg_radiation_per_hour[8], '15_hor': avg_radiation_per_hour[9], '16_hor': avg_radiation_per_hour[10], 
                                  '17_hor': avg_radiation_per_hour[11], '18_hor': avg_radiation_per_hour[12],'6_pv': avg_radiation_per_hour2[0], '7_pv': avg_radiation_per_hour2[1], 
                                  '8_pv': avg_radiation_per_hour2[2], '9_pv': avg_radiation_per_hour2[3], '10_pv': avg_radiation_per_hour2[4], 
                                  '11_pv': avg_radiation_per_hour2[5], '12_pv': avg_radiation_per_hour2[6], '13_pv': avg_radiation_per_hour2[7], 
                                  '14_pv': avg_radiation_per_hour2[8], '15_pv': avg_radiation_per_hour2[9], '16_pv': avg_radiation_per_hour2[10], 
                                  '17_pv': avg_radiation_per_hour2[11], '18_pv': avg_radiation_per_hour2[12]}, 
                                 ignore_index=True)

# 合并结果并保存到一个CSV文件中
result_merged = pd.merge(result1, result_df, on='sl_id')
result_merged.to_csv( r'D:\paper\01_studyarea\03_太阳能辐射估计\03_02_跑全部的结果\5_所有点\folder_all\right_average.csv', index=False)


# 7.合并所有csv

In [15]:
import os
import pandas as pd
from tqdm import tqdm  # 正确导入 tqdm 库中的 tqdm 函数

# 文件夹路径
folder_path = r'D:\paper\01_studyarea\03_太阳能辐射估计\03_02_跑全部的结果\5_所有点\folder_all\average'  # 替换为你的文件夹路径

# 获取文件夹中所有的CSV文件名
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# 确保文件按照文件名的自然数顺序排序
csv_files.sort(key=lambda x: int(x.split('_')[0]))

# 初始化一个空的DataFrame来存储合并后的数据
combined_df = pd.DataFrame()

# 使用tqdm显示进度条
with tqdm(total=len(csv_files)) as pbar:
    # 逐个读取和合并CSV文件
    for csv_file in csv_files:
        file_path = os.path.join(folder_path, csv_file)
        df = pd.read_csv(file_path)  # 读取CSV文件
        
        # 合并到总的DataFrame中
        combined_df = pd.concat([combined_df, df], ignore_index=True)
        
        # 更新进度条
        pbar.update(1)

# 将合并后的DataFrame保存为一个新的CSV文件，只保留一个header
output_file = os.path.join(folder_path, 'combined.csv')  # 输出文件路径为文件夹下的 combined.csv
combined_df.to_csv(output_file, index=False)

print(f"合并后的文件已保存至 {output_file}")




100%|██████████| 13/13 [00:00<00:00, 148.00it/s]


合并后的文件已保存至 D:\paper\01_studyarea\03_太阳能辐射估计\03_02_跑全部的结果\5_所有点\folder_all\average\combined.csv


# 8.替换掉个别不对的点的结果

In [20]:
import pandas as pd

# 读取第一个CSV文件，包含要替换的ID
csv1 = pd.read_csv(r'D:\paper\01_studyarea\03_太阳能辐射估计\03_02_跑全部的结果\5_所有点\folder_all\right_average.csv', encoding='GBK',sep=',')

# 读取第二个CSV文件，包含被替换的sl_id列
csv2 = pd.read_csv(r'D:\paper\01_studyarea\03_太阳能辐射估计\03_02_跑全部的结果\5_所有点\folder_all\average\combined.csv', encoding='GBK',sep=',')

# 遍历csv1的每一行，替换csv2中对应的行
for index, row in csv1.iterrows():
    id_value = row['sl_id']
    # 找到csv2中匹配sl_id的行，并用csv1的行数据替换它
    csv2.loc[csv2['sl_id'] == id_value, csv2.columns] = row.values

# 保存结果到一个新的CSV文件中
csv2.to_csv(r'D:\paper\01_studyarea\03_太阳能辐射估计\03_02_跑全部的结果\5_所有点\folder_all\average\combined2.csv', index=False)

print("替换操作完成，结果已保存到'test2.csv'")

替换操作完成，结果已保存到'test2.csv'
